In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [2]:
url = 'https://www.giants.com/team/stats/'
response = requests.get(url)

In [3]:
print("Response:", response.status_code,response.url) #200 means it went through


Response: 200 https://www.giants.com/team/stats/


In [4]:
giants_stats = BeautifulSoup(response.text,'html.parser')
giants_stats_body = giants_stats.body
nfl_div = giants_stats_body.find_all('div')
giants_stats_body_main_content = giants_stats_body.find(id="main-content")
print(giants_stats_body_main_content.prettify())

pan>
           <span>
            187
           </span>
           <span>
            35
           </span>
          </div>
          <div class="nfl-o-team-h2h-stats__label">
           <span class="nfl-o-team-h2h-stats__label--full">
            FIRST DOWNS
           </span>
           <span class="nfl-o-team-h2h-stats__label--short">
            1ST DOWNS
           </span>
           <span class="nfl-o-team-h2h-stats__label--first-child nfl-o-team-h2h-stats__label--child">
            Rushing
           </span>
           <span class="nfl-o-team-h2h-stats__label--child">
            Passing
           </span>
           <span class="nfl-o-team-h2h-stats__label--last-child nfl-o-team-h2h-stats__label--child">
            By Penalty
           </span>
          </div>
          <div class="nfl-o-team-h2h-stats__value">
           <span>
            100
           </span>
           <span>
            211
           </span>
           <span>
            32
           </span>
     

In [5]:
offensive_passing_table = giants_stats_body_main_content.find("div", {"class": "nfl-o-teamstats"})
print(offensive_passing_table.prettify())

<div class="nfl-o-teamstats">
 <h3 class="nfl-o-teamstats__title">
  Passing
 </h3>
 <div class="d3-o-table--horizontal-scroll">
  <table aria-hidden="true" class="d3-o-table d3-o-table--row-striping d3-o-table--sortable {sortlist: [[1,1], [2,1]], sortinitialorder: 'desc'}" data-require='"' modules="" tablesortable&quot;="">
   <thead>
    <tr>
     <th>
      Player
     </th>
     <th scope="col">
      ATT
     </th>
     <th scope="col">
      COMP
     </th>
     <th scope="col">
      YDS
     </th>
     <th scope="col">
      COMP%
     </th>
     <th scope="col">
      YDS/ATT
     </th>
     <th scope="col">
      TD
     </th>
     <th scope="col">
      TD%
     </th>
     <th scope="col">
      INT
     </th>
     <th scope="col">
      INT%
     </th>
     <th scope="col">
      LONG
     </th>
     <th scope="col">
      SCK
     </th>
     <th scope="col">
      SCK/LOST
     </th>
     <th scope="col">
      RATE
     </th>
    </tr>
   </thead>
   <tbody>
    <tr aria-

In [6]:
headers = offensive_passing_table.find('thead')
print(headers)

<thead>
<tr>
<th>Player</th>
<th scope="col">ATT</th>
<th scope="col">COMP</th>
<th scope="col">YDS</th>
<th scope="col">COMP%</th>
<th scope="col">YDS/ATT</th>
<th scope="col">TD</th>
<th scope="col">TD%</th>
<th scope="col">INT</th>
<th scope="col">INT%</th>
<th scope="col">LONG</th>
<th scope="col">SCK</th>
<th scope="col">SCK/LOST</th>
<th scope="col">RATE</th>
</tr>
</thead>


In [7]:
clean_headers = headers.find_all('th')
print(clean_headers)

[<th>Player</th>, <th scope="col">ATT</th>, <th scope="col">COMP</th>, <th scope="col">YDS</th>, <th scope="col">COMP%</th>, <th scope="col">YDS/ATT</th>, <th scope="col">TD</th>, <th scope="col">TD%</th>, <th scope="col">INT</th>, <th scope="col">INT%</th>, <th scope="col">LONG</th>, <th scope="col">SCK</th>, <th scope="col">SCK/LOST</th>, <th scope="col">RATE</th>]


In [8]:
columnHeader = []

for c in clean_headers:
    columnHeader.append(c.text)

print(columnHeader)

['Player', 'ATT', 'COMP', 'YDS', 'COMP%', 'YDS/ATT', 'TD', 'TD%', 'INT', 'INT%', 'LONG', 'SCK', 'SCK/LOST', 'RATE']


In [92]:
clean_offensive_passing_stats = offensive_passing_table.find_all('tr',{"data-loadmore":"Passing"})

# clean_offensive_passing_stats

# rowTable = clean_offensive_passing_stats.find('td')

In [132]:
rowData = []


for r in clean_offensive_passing_stats:
    rowData.append(r.text)
    # str(rowData).split


rowData = [r.strip('\n') for r in rowData]
clean_rowData = str(rowData).split('\n')
# clean_rowData


# print(rowData)
# print()
# rowData = [r.strip('\n') for r in rowData]
# print()
# print(rowData)
# rowData = [r.strip('\n\n') for r in rowData]
# print()
# print(rowData)

